# Extended Resolution Analysis with MSSR and Manders Coefficient Calculation

This Jupyter notebook provides a detailed workflow for conducting extended resolution analysis using the MSSR (Mean Shift Super Resolution) approach on fluorescence microscopy images, followed by Manders' coefficient calculation to quantify channel colocalization. The process leverages the `napari-superres` library for enhanced resolution, operating on OME-Zarr files containing both fluorescence imaging data and essential metadata extracted from `.oib` files. Regions of Interest (ROIs) are defined manually in ImageJ and saved for batch processing, facilitating high-throughput colocalization studies.

### Reference:
This methodology is based on the approach outlined in:  
**Torres E. et al.**, "Extending resolution within a single imaging frame," *Nature Communications*, 2022. doi:10.1038/s41467-022-34693-9

**Script Author:** Adan Guerrero  
**Contact:** adan.guerrero@ibt.unam.mx

---

### Workflow Overview

1. **Resolution Extension (MSSR):**  
   - The raw fluorescence images undergo MSSR processing to achieve enhanced resolution. The MSSR technique helps resolve finer details within a single imaging frame, especially useful for high-resolution microscopy.

2. **Manders' Coefficient Calculation:**  
   - Following MSSR processing, Manders' coefficients are calculated to assess colocalization between specific fluorescence channels, facilitating an in-depth analysis of spatial relationships among cellular components.

3. **Saving Results:**  
   - Analysis results, including computed Manders' coefficients, are saved in a CSV file (i.e. `Toxin_vs_Mito_all_manders_results.csv`). This CSV file enables easy access for reloading data and further statistical analysis or validation.

---

### Experimental Design

The experimental setup focuses on analyzing the colocalization of toxins with major cellular structures through different fluorescence channels, captured on an Olympus FV1000 microscope with a 60x objective at 60 nm per pixel resolution. Specific configurations were established for different cellular structures and components, as detailed below.

- **Image Channels:**  
  The microscopy images contain four channels:
    - **C1 (Blue):** DAPI (nucleus)
    - **C2 (Cyan/Green):** Specific markers (e.g., toxin, ER)
    - **C3 (Red):** Markers like actin or other cytoskeletal elements
    - **C4 (Gray):** Additional markers (e.g., Mitochondria, Endosomes)

- **Regions of Interest (ROIs):**  
   ROIs measuring \(5 \times 5 \, \mu \text{m}^2\) were manually defined using the rectangle tool in FIJI (ImageJ v1.54f) and saved as a `.zip` file (`Analyze → Tools → ROI Manager → More → Save`). These ROIs allow focused analysis of specific areas for colocalization.

### Data Structure

The results are saved in `Toxin_vs_Mito_all_manders_results.csv` with the following columns:

- **roi**: Identifier for each ROI, detailing the specific image region analyzed.
- **m1**: Manders' coefficient for one channel, representing colocalization with a specific component (e.g., toxin).
- **m2**: Manders' coefficient for the second channel, representing colocalization with another component (e.g., mitochondria).
- **sample**: Sample identifier, useful for distinguishing different experimental conditions or replicates.

Each row provides the colocalization results for an individual ROI, allowing for statistical evaluation of colocalization across multiple samples. This structure supports batch analysis for examining experimental variations in colocalization among different cellular structures.

### Colocalization Comparisons:

1. **Actin Channels:**
   - **C1 + C2:** Toxin and nucleus (DAPI)
   - **C2 + C3:** Toxin and actin, assessing cytoskeletal interactions
   - **Additional Channels:** Similar analysis for lysosomal and mitochondrial markers

2. **Endoplasmic Reticulum (ER) Channels:**
   - **C1 + C3:** Toxin and nucleus
   - **C2 + C3:** ER and toxin colocalization

3. **Endosomal Channels:**
   - **C1 + C2:** Toxin and nucleus
   - **C2 + C4:** Toxin and endosomes

Additional colocalization analyses focus on cytoplasmic, extracellular and nuclear regions, with vesicular markers (ER, Endosomes, Lysosomes, and Mitochondria) studied in their colocalization with toxins, enabling insights into cellular dynamics and toxin distribution.

--- 


## Define directory containing the files to be analyzed

In [1]:
directory_path = '/home/jovyan/LNMA/bravoa/data/New2 Fig para colocalizacion Manders-Mito'
order_types = ['Cit']
ch_m1 = 1
ch_m2 = 2
df_name = 'Toxin_vs_Mito'

directory_path = '/home/jovyan/LNMA/bravoa/data/Seleccion ROIs circulares RE'
order_types = ['VExt']
ch_m1 = 2
ch_m2 = 1
df_name = 'Vesicles_Toxin_vs_ER_circular'

## Define Parameter

In [2]:
### Parameters for MSSR

fwhm = 8  # Full Width at Half Maximum
amp = 1  # Amplitude
order = 1  # Order
mesh = True  # Optional, default is True
ftI = False  # Optional, default is False
intNorm = False  # Optional, default is True
sigma = 2 # For the Gassian filter
visualization_percentiles=(2.5, 97.5)

## Install Libraries

In [3]:
print("Installing necessary libraries...")
!pip install os ome-zarr PyQt5 napari[all] > /dev/null 2>&1
!pip install natsort  > /dev/null 2>&1
!pip install git+https://github.com/RoccoDAnt/napari-superres.git > /dev/null 2>&1
print("Libraries installed successfully.")

Installing necessary libraries...
Libraries installed successfully.


## Import libraries

In [9]:
import zarr
import os
import pandas as pd
from natsort import natsorted
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import gaussian
from napari_superres.core_mssr import mssr_class
mssr_instance = mssr_class()

## Define functions

In [5]:
# Extract the rectangular region from the specified channel
def extract_rect_roi(image, left, top, width, height):
    return image[top:top+height, left:left+width]

# Extract the oval region from the image, covering the full image dimensions
def extract_oval_roi_from_image(image):
    # Get image dimensions
    height, width = image.shape
    
    # Calculate the center of the oval
    cx, cy = width // 2, height // 2
    
    # Generate a coordinate grid based on the image dimensions
    y, x = np.ogrid[:height, :width]
    
    # Create the oval mask centered within the full image
    mask = ((x - cx) ** 2) / (width / 2) ** 2 + ((y - cy) ** 2) / (height / 2) ** 2 <= 1
    
    # Apply the mask to the image
    oval_roi = np.where(mask, image, 0)
    
    return oval_roi

def manders_colocalization(image1, image2):
    overlap = np.logical_and(image1 > 0, image2 > 0)
    m1 = np.sum(overlap) / np.sum(image1 > 0) if np.sum(image1 > 0) > 0 else 0
    m2 = np.sum(overlap) / np.sum(image2 > 0) if np.sum(image2 > 0) > 0 else 0
    return m1, m2
    
def normalize_channel(channel, lower_percentile, upper_percentile):
    lower_bound = np.percentile(channel, lower_percentile)
    upper_bound = np.percentile(channel, upper_percentile)
    channel = np.clip(channel, lower_bound, upper_bound)
    # Avoid division by zero
    if upper_bound == lower_bound:
        return np.zeros_like(channel)
    return (channel - lower_bound) / (upper_bound - lower_bound)

## Load and analyze the OME-Zarr files

In [1]:
# Process each file in the specified directory

M = []
for file_name in natsorted(os.listdir(directory_path)):
    # Construct the original full path
    if file_name.endswith('.zarr'):
        ome_zarr_path = os.path.join(directory_path, file_name)
        #print(ome_zarr_path)
        
        ## Load the OME-Zarr file
        images = zarr.open(ome_zarr_path, mode="r")
        channel_m1 = images["image_data"][0, ch_m1, 0, :, :].astype(np.float32)
        channel_m2 = images["image_data"][0, ch_m2, 0, :, :].astype(np.float32)

        ## Extract ROI metadata from the OME-Zarr file
        roi_metadata = images.attrs.get("roi_metadata", {})
        roi_metadata = [item for item in roi_metadata if any(order_type in item['name'] for order_type in order_types)]
        
        for roi_info in roi_metadata:

            left = roi_info['left']
            if(left < 0):
                left = 0
            top = roi_info['top']
            if(top < 0):
                top = 0
            width = roi_info['width']
            height = roi_info['height']

            # Extract ROIs
            roi_channel_m1 = extract_rect_roi(channel_m1, left, top, width, height)
            roi_channel_m2 = extract_rect_roi(channel_m2, left, top, width, height)
    
            # Apply Gaussian filter
            roi_channel_m1 = gaussian(roi_channel_m1, sigma)
            roi_channel_m2 = gaussian(roi_channel_m2, sigma)

            # Apply MSSR
            roi_channel_m1_mssr = mssr_instance.sfMSSR(img=roi_channel_m1,
                                               fwhm=fwhm, amp=amp,
                                               order=order,
                                               mesh=mesh, ftI=ftI,
                                               intNorm=intNorm)
    
            roi_channel_m2_mssr = mssr_instance.sfMSSR(img=roi_channel_m2,
                                               fwhm=fwhm, amp=amp,
                                               order=order,
                                               mesh=mesh, ftI=ftI,
                                               intNorm=intNorm)
            if(roi_info['type']=='oval'):
                roi_channel_m1_mssr = extract_oval_roi_from_image(roi_channel_m1_mssr)
                roi_channel_m2_mssr = extract_oval_roi_from_image(roi_channel_m2_mssr)

            # Compute Manders' colocalization coefficients after MSSR
            m1, m2 = manders_colocalization(roi_channel_m1_mssr, roi_channel_m2_mssr)

            # Extract the number before the first underscore
            sample_number_str = file_name.split('_')[0]

            # Convert to an integer
            sample_number = int(sample_number_str)  # or float(number_str) if it might be a decimal


            # Append the results as a tuple
            M.append((roi_info['name'], round(m1, 3), round(m2, 3), sample_number))

SyntaxError: 'return' outside function (1590367923.py, line 73)

In [7]:
# Specify headers when creating the DataFrame
df = pd.DataFrame(M, columns=['roi', 'm1', 'm2', 'sample'])
# Save the DataFrame
df_file_name = f"{df_name}_all_manders_results.csv"
df_file_path = os.path.join(directory_path, df_file_name)
df.to_csv(df_file_path, index=False)
df

,roi,m1,m2,sample
0,0002-0155-0077-VExt,0.402,0.392,1
1,0002-0098-0200-VExt,0.541,0.492,1
2,0002-0070-0320-VExt,0.427,0.416,1
3,0002-0082-0394-VExt,0.555,0.419,1
4,0002-0127-0476-VExt,0.568,0.447,1
...,...,...,...,...
105,0002-0515-0944-VExt,0.505,0.499,15
106,0002-0591-0942-VExt,0.530,0.522,15
107,0002-0939-0143-VExt,0.513,0.508,15
108,0002-0711-0186-VExt,0.634,0.560,15
